In [2]:
import openeo
import pystac
from datetime import datetime

In [3]:
connection = openeo.connect("https://openeo.vito.be")
# connection = openeo.connect("openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.vito.be/openeo/1.2/' with OidcBearerAuth>

## Create example GTiff
First we download a simple GTiff file containing ndvi data from an openeo provider. This file will be used as an example for the following steps.

In [3]:
bbox = {"west": 5.0, "south": 51.2, "east": 5.1, "north": 51.3}
collection = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent=bbox,
    temporal_extent=["2021-06-01", "2021-06-30"],
    bands=["B04", "B08", "SCL"],
    max_cloud_cover=95
)
collection = collection.process("mask_scl_dilation", scl_band_name="SCL", data=collection)
collection = collection.ndvi("B04", "B08", target_band="NDVI-band").filter_bands("NDVI-band")
collection.download("ndvi-file.tiff", format="GTiff")

## Create a STAC item
We create a STAC item for the GTiff file using pystac.

A STAC item is a JSON file that describes a geospatial dataset. It contains metadata about the dataset, such as the spatial extent, the time range, the bands, and the projection. The STAC item also contains links to the data files in assets.

It is important that the GTiff file is accessible for the openeo backend. This can be done by uploading the file to a cloud storage service like AWS S3 or Google Cloud Storage, or by storing the file in your public directory on a terrascope uservm (when using the openeo.vito.be backend).

Make sure the band names are set correctly in the STAC item. In this case the band name is "NDVI-band".

In [5]:
pystac.Item(
    id="ndvi-example-stac-item",
    geometry={
        "type": "Polygon",
        "coordinates": [[5.0, 51.2],[5.1, 51.2],[5.1, 51.3],[5.0, 51.3],[5.0, 51.2]]
    },
    bbox=[5.0, 51.2, 5.1, 51.3],
    datetime=datetime.fromisoformat("2021-06-01"),
    properties={ # These properties are optional, but can speed up the loading of the data.
        # They can be extracted from the GTiff file using e.g. qgis
        "proj:epsg": 32631,
        "proj:shape": [1133,730], # Caveat: this is [height, width] and not [width, height]
        "proj:bbox": [639420,5673960,646720,5685290]
    },
    stac_extensions=["eo", "proj"],
    assets={
        "ndvi": pystac.Asset(
            href="https://artifactory.vgt.vito.be/artifactory/auxdata-public/openeo/load-stac-example/ndvi-file.tiff", # public path/URL to the file
            title="Normalized Difference Vegetation Index",
            extra_fields={
                "eo:bands": [ # it is necessary to define the bands in the eo extension for openeo to be able to load it
                    {
                        "name": "NDVI-band",
                        "common_name": "ndvi",
                    }
                ],
            }
        )
    }
).save_object(dest_href="ndvi-example-stac-item.json", include_self_link=True)

## Load the STAC item in openeo as a collection
We load the STAC item in openeo as a collection using `load_stac` and download the result as a GTiff file.

IMPORTANT: Make sure the STAC item and the GTiff are accessible for the openeo backend. This can be done by uploading the STAC item to a cloud storage service like AWS S3 or Google Cloud Storage, or by storing the STAC item in your public directory on a terrascope uservm (when using the openeo.vito.be backend).

In [6]:
datacube = connection.load_stac("https://artifactory.vgt.vito.be/artifactory/auxdata-public/openeo/load-stac-example/ndvi-example-stac-item.json")
datacube.download("ndvi-from-stac.tiff", format="GTiff")